In [1]:
using DataStructures
using LinearAlgebraicRepresentation
Lar = LinearAlgebraicRepresentation
using BenchmarkTools

In [2]:
function hollowCyl(r=1., R=2., height=6., angle=2*pi)
    function hollowCyl0(shape=[36, 1, 1])
        V, CV = cuboidGrid(shape)
        V = [angle/shape[1] 0 0;0 (R-r)/shape[2] 0;0 0 height/shape[3]]*V
        V = broadcast(+, V, [0, r, 0])
        W = [V[:, k] for k=1:size(V, 2)]
        V = hcat(map(p->let(u, v, z)=p;[v*cos(u);v*sin(u);z] end, W)...)
        W, CW = simplifyCells(V, CV)
        return W, CW
    end
    return hollowCyl0
end

hollowCyl (generic function with 5 methods)

In [3]:
@code_warntype Lar.hollowCyl()();

Variables
  #self#::LinearAlgebraicRepresentation.var"#hollowCyl0#318"{Float64,Float64,Float64,Float64}

Body::Tuple{Array,Array{Array{Int64,1},1}}
1 ─ %1 = Base.vect(36, 1, 1)::Array{Int64,1}
│   %2 = (#self#)(%1)::Tuple{Array,Array{Array{Int64,1},1}}
└──      return %2


In [4]:
@btime W,CW = Lar.hollowCyl()();

  359.081 μs (4280 allocations: 411.70 KiB)


## Uso della macro @threads 

#### Per settare il numero di threads pari a 2 ho dovuto per forza eseguire (come scritto nella guida di julia) i seguenti passaggi : export JULIA_NUM_THREADS=2 , set JULIA_NUM_THREADS=2 , il numero di threads va settato in base alla macchina detenuta , se non lo si fa l'uso dei threads è nullo .

In [6]:
using Base.Threads
Threads.nthreads() = 2  #setto il numero di threads pari a 2
nthreads()

2

In [21]:
function hollowCyl1(r=1., R=2., height=6., angle=2*pi)
    function hollowCyl01(shape=[36, 1, 1])
        V, CV = Lar.cuboidGrid(shape)
        V = [angle/shape[1] 0 0;0 (R-r)/shape[2] 0;0 0 height/shape[3]]*V
        V = broadcast(+, V, [0, r, 0])
        n = size(V,2)
        W = Array{Array{Float64,1}}(undef, n)
        
        @inbounds @threads for k=1:n
         W[k] = V[:,k]
    end
        V = hcat(map(p->let(u, v, z)=p;[v*cos(u);v*sin(u);z] end, W)...)
        W, CW = Lar.simplifyCells(V, CV)
        return W, CW
    end
    return hollowCyl01
end

hollowCyl1 (generic function with 5 methods)

In [22]:
@btime W,CW = hollowCyl1()();

  361.098 μs (4282 allocations: 411.77 KiB)


Ho cercato di utilizzare i threads quindi per farlo ho dovuto "esternare" il ciclo for , cosi facendo ho potuto usare la macro @threads . Di fatti i cambiamenti non sono efficienti e spesso meno comprensibili e piu disordinati ma solo in questo modo sono state possibili applicare le macro . Senza l'uso delle macro , con il seguente ciclo for ho notato che la velocità di esecuzione aumenta di molto ( si puo notare nel notebook "disk" , nel caso di disk ho deciso di non omettere nessun passaggio e far vedere tutta l'evoluzione del codice stampandone i risultati legati alla velocità di esecuzione . ) Questo messaggio è comune e sarà riportato alla fine di ogni notebook .